In [177]:
%load_ext autoreload
%autoreload 2


from util import *
from pastalgorithm import algorithm_cost,algorithm_GA
from myalgorithm import algorithm_basic_greedy,algorithm_regret,algorithm_basic_greedy_v2
from util_zoe.myalgorithm import algorithm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

problem_file = './stage1_problem/STAGE1_2.json'
timelimit = 60

# np.random.seed(1)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

alg_start_time = time.time()

exception = None

solution = None

In [77]:
alpha = 0.1
beta = 0.2
gamma = 0.3
dist_max = np.max(DIST)
volumes = [order.volume for order in ALL_ORDERS]
v_max = max(volumes)
ready_times = [order.ready_time for order in ALL_ORDERS]
r_max = max(ready_times)
deadlines = [order.deadline for order in ALL_ORDERS]
d_max = max(deadlines)

N = len(ALL_ORDERS)
relatedness = np.zeros((N, N))

# Calculate the values and store them in the result matrix
for i in range(N):
    for j in range(N):
        relatedness[i, j] = (
            alpha*(DIST[i, j]/dist_max + DIST[i + K, j + K]/dist_max)
            + beta*(ALL_ORDERS[i].volume/v_max - ALL_ORDERS[j].volume/v_max)
            + gamma*(abs(ALL_ORDERS[i].ready_time/r_max - ALL_ORDERS[j].ready_time/r_max) \
                     + abs(ALL_ORDERS[i].deadline/d_max - ALL_ORDERS[j].deadline/d_max))
        )

In [174]:
q=4
p=1
D = []
D.append(np.random.randint(0, K)) 
D.append(np.random.randint(0, K)) 
while len(D)<q:
    r = random.choice(D)
    L = relatedness[r]
    sorted_indices = np.argsort(L)
    y = np.random.uniform(0, 1)**p
    y = int(np.floor(y*len(L))) 
    D.append(sorted_indices[y]) # y번째로 작은 값의 인덱스

In [175]:
D

[11, 22, 97, 30]

In [ ]:

def check_crossover_feasibility(new_shop_seq,riders_dict,ALL_ORDERS,DIST,K):

    reduced_cost = {'CAR':99999,'BIKE':99999,'WALK':99999}
    optimal_bundle = {'CAR':None,'BIKE':None,'WALK':None}
    riders = ['CAR','BIKE','WALK']
    total_volume = get_total_volume(ALL_ORDERS, new_shop_seq)
    for rider_type in ['CAR','BIKE','WALK']:
        if total_volume > riders_dict[rider_type].capa :
            riders.remove(rider_type)
    if len(riders) == 0:
        return None,riders_dict

    for shop_pem in permutations(new_shop_seq):
        for dlv_pem in permutations(new_shop_seq):
            for rider_type in riders:
                rider = riders_dict[rider_type]
                if rider.available_number<=0:
                    continue
                feasibility_check = test_feasibility(ALL_ORDERS, rider, shop_pem, dlv_pem)
                if feasibility_check:
                    total_dist = get_total_distance(K, DIST, shop_pem, dlv_pem)

                    new_bundle = Bundle(ALL_ORDERS, rider, list(shop_pem), list(dlv_pem), total_volume, total_dist)
    #                 riders_dict[rider_type].available_number -=1
    #                 return new_bundle,riders_dict
    # return None, riders_dict
                    # if len(new_shop_seq)>3:
                    #     riders_dict[rider_type].available_number -=1
                    #     return new_bundle,riders_dict
                    
                    if new_bundle.cost < reduced_cost[rider_type]:
                        optimal_bundle[rider_type] = new_bundle
                        reduced_cost[rider_type] = new_bundle.cost 
                       

    if reduced_cost['CAR'] < 99999 or reduced_cost['BIKE']  < 99999 or reduced_cost['WALK']  < 99999:
        max_reduced_cost = min(reduced_cost.values())
        max_rider = [k for k, v in reduced_cost.items() if v == max_reduced_cost]
        riders_dict[max_rider[0]].available_number-=1
        return optimal_bundle[max_rider[0]],riders_dict
    else:
        return None ,riders_dict   

In [176]:
import copy
from util_past.util_GA import *
# Assume solution, D, and bundle.shop_seq are lists or sets
my_solution = copy.deepcopy(solution)
print('total length:', len(my_solution))

to_remove = []  # 임시 리스트 생성

for bundle in my_solution:
    intersection = list(set(D) & set(bundle.shop_seq))
    if len(intersection) != 0:
        to_remove.append(bundle)  # 삭제할 요소를 임시 리스트에 추가
        D = list(set(D) - set(intersection))
        new_shop_seq = list(set(bundle.shop_seq) - set(intersection))
        new_bundle,riders_dict = check_crossover_feasibility(new_shop_seq,riders_dict,ALL_ORDERS,DIST,K)
        print(D)
# 임시 리스트에 모인 요소들을 한꺼번에 삭제

print(to_remove)
for bundle in to_remove:
    my_solution.remove(bundle)

print(D)
print('total length:', len(my_solution))


total length: 33


NameError: name 'riders_dict' is not defined

In [193]:

problem_file = './stage1_problem/STAGE1_2.json'
timelimit = 60

# np.random.seed(1)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

alg_start_time = time.time()

exception = None

solution = None
solution = algorithm_basic_greedy_v2(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

Best obj = 7244.27
0 th current_solution 7164.422
10 th current_solution 6445.262
20 th current_solution 5793.428000000001
30 th current_solution 5178.84
40 th current_solution 4587.652
50 th current_solution 4039.9879999999994
60 th current_solution 3564.2260000000006
D before iteration: [17, 44, 64, 27]
[Bundle(all_orders, CAR, [47, 42, 64], [42, 47, 64], 79, True), Bundle(all_orders, BIKE, [27, 32, 6, 18], [18, 32, 27, 6], 86, True), Bundle(all_orders, BIKE, [8, 7, 14, 44], [7, 8, 14, 44], 93, True), Bundle(all_orders, BIKE, [30, 17, 40, 29], [17, 30, 29, 40], 98, True)]
D after iteration: []


In [197]:
updated_bundles = solution
solution_2 = [
            [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
            for bundle in updated_bundles
                ]
checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution_2)
    

In [198]:
checked_solution

{'total_cost': 344230.60000000003,
 'avg_cost': 3442.3060000000005,
 'num_drivers': 35,
 'total_dist': 137943,
 'feasible': True,
 'infeasibility': None,
 'bundles': [['CAR', [73], [73]],
  ['CAR', [71, 90, 92], [90, 71, 92]],
  ['BIKE', [69, 96, 97], [96, 97, 69]],
  ['BIKE', [68, 55, 79, 84], [68, 84, 79, 55]],
  ['CAR', [98, 76, 70], [70, 98, 76]],
  ['CAR', [10, 9, 15, 31], [15, 10, 9, 31]],
  ['CAR', [49, 63, 74], [63, 49, 74]],
  ['CAR', [67, 80, 59], [67, 80, 59]],
  ['BIKE', [61, 95, 66], [95, 66, 61]],
  ['CAR', [36, 34], [36, 34]],
  ['CAR', [87, 86, 75], [87, 75, 86]],
  ['CAR', [54, 81, 62, 58], [54, 62, 81, 58]],
  ['BIKE', [82, 28, 99], [82, 28, 99]],
  ['CAR', [43, 37, 41], [37, 43, 41]],
  ['CAR', [38, 51], [51, 38]],
  ['BIKE', [56, 77, 91], [77, 91, 56]],
  ['BIKE', [78, 94, 50, 46], [46, 94, 50, 78]],
  ['CAR', [21, 1, 2], [2, 21, 1]],
  ['BIKE', [85, 65, 53], [65, 53, 85]],
  ['BIKE', [20, 16, 11], [11, 16, 20]],
  ['BIKE', [25, 83], [25, 83]],
  ['CAR', [3, 22, 23]

## 개별 문제 실행

In [227]:

problem_file = './stage1_problem/STAGE1_2.json'
timelimit = 60

# np.random.seed(1)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

alg_start_time = time.time()

exception = None

solution = None
try:
    # Run algorithm!
    solution = algorithm_basic_greedy_v2(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)
except Exception as e:
    exception = f'{e}'


alg_end_time = time.time()

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

checked_solution['time'] = alg_end_time - alg_start_time
checked_solution['timelimit_exception'] = (alg_end_time - alg_start_time) > timelimit + 1 # allowing additional 1 second!
checked_solution['exception'] = exception

checked_solution['prob_name'] = prob['name']
checked_solution['prob_file'] = problem_file

checked_solution

Best obj = 7244.27
0 th current_solution 7164.422
10 th current_solution 6445.262
20 th current_solution 5793.428000000001
30 th current_solution 5178.84
40 th current_solution 4587.652
50 th current_solution 4039.9879999999994
60 th current_solution 3564.2260000000006
performance before local search : 3351.6419999999994
D: [48, 23, 88, 1]
before remove: {'BIKE': Rider([BIKE, 5.291005291005291, 100, 60, 8000, 120, 6]), 'WALK': Rider([WALK, 1.3227513227513228, 70, 30, 8000, 120, 30]), 'CAR': Rider([CAR, 4.2328042328042335, 200, 100, 6000, 180, 82])}
[Bundle(all_orders, CAR, [21, 1, 2], [2, 21, 1], 78, True), Bundle(all_orders, CAR, [3, 22, 23], [23, 3, 22], 90, True), Bundle(all_orders, CAR, [48, 93, 57, 88], [48, 88, 57, 93], 183, True)]
CAR
CAR
CAR
after remove: {'BIKE': Rider([BIKE, 5.291005291005291, 100, 60, 8000, 120, 6]), 'WALK': Rider([WALK, 1.3227513227513228, 70, 30, 8000, 120, 30]), 'CAR': Rider([CAR, 4.2328042328042335, 200, 100, 6000, 180, 85])}
Bundle(all_orders, CAR, [21,

{'feasible': False,
 'infeasibility': 'Solution must be a list of bundle information!',
 'bundles': None,
 'time': 5.301474094390869,
 'timelimit_exception': False,
 'exception': "'NoneType' object has no attribute 'shop_seq'",
 'prob_name': 'STAGE1_2',
 'prob_file': './stage1_problem/STAGE1_2.json'}

In [ ]:
draw_route_solution(ALL_ORDERS, checked_solution)

In [ ]:
draw_bundle_solution(ALL_ORDERS, ALL_RIDERS, DIST, checked_solution)

## 전체 문제 실행

In [34]:
# baseline='avg_cost': 4903.952,
# 'avg_cost': 4118.8099999999995,
# problem_file = 'STAGE1_1.json'
avg_cost = []
for i in range(1,19):

    problem_file = f'./stage1_problem/STAGE1_{i}.json'
    timelimit = 60

    # np.random.seed(1)

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST']) # 100*100 주문,도착지 pair가 order 50개에 들어있음.
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time) # T라는 속성에 커스텀 거리 행렬 들어감.

    alg_start_time = time.time()

    exception = None

    solution = None
    try:
        # Run algorithm!
        solution = algorithm_basic_greedy(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)
    except Exception as e:
        exception = f'{e}'


    alg_end_time = time.time()
    print(f'total elapsed time : {alg_end_time-alg_start_time}')
    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time)

    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)
    avg_cost.append((K,checked_solution['avg_cost'],alg_end_time-alg_start_time))

Best obj = 7322.13
0 th current_solution 7168.21
10 th current_solution 6463.361999999999
20 th current_solution 5797.816
30 th current_solution 5170.696
40 th current_solution 4592.556
50 th current_solution 4092.7380000000003
end iteration
current_solution 3826.104000000001
total elapsed time : 7.7468461990356445
Best obj = 7244.27
0 th current_solution 7085.432
10 th current_solution 6376.682
20 th current_solution 5731.4980000000005
30 th current_solution 5118.414000000001
40 th current_solution 4530.642
50 th current_solution 3988.9479999999994
60 th current_solution 3521.646
end iteration
current_solution 3335.3719999999994
total elapsed time : 8.560051202774048
Best obj = 7358.805
0 th current_solution 7267.803000000001
10 th current_solution 6834.755000000001
20 th current_solution 6425.569
30 th current_solution 6027.781000000001
40 th current_solution 5641.607500000002
50 th current_solution 5268.146500000002
60 th current_solution 4905.536500000002
70 th current_solution 455

## 주문 수 별 평균 계산

In [36]:

# 100일 때와 200일 때의 avg_cost 합계와 개수를 계산
sum_100 = sum_200 = count_100 = count_200 = count_300 = sum_300 = 0

for k, cost,_ in avg_cost:
    if k == 100:
        sum_100 += cost
        count_100 += 1
    elif k == 200:
        sum_200 += cost
        count_200 += 1
    elif k == 300:
        sum_300 += cost
        count_300 += 1


# 평균 계산
avg_100 = sum_100 / count_100 if count_100 != 0 else 0
avg_200 = sum_200 / count_200 if count_200 != 0 else 0

avg_300 = sum_300 / count_300 if count_300 != 0 else 0

# 결과를 리스트에 추가
avg_cost.append(('mean',100, avg_100))
avg_cost.append(('mean',200, avg_200))

avg_cost.append(('mean',300, avg_300))

avg_cost

[(100, 3826.104000000001, 7.7468461990356445),
 (100, 3335.3719999999994, 8.560051202774048),
 (200, 3427.574499999999, 26.44631314277649),
 (200, 2761.9255000000003, 37.296088457107544),
 (300, 3491.021666666668, 59.31600999832153),
 (300, 3387.4366666666674, 59.33166837692261),
 (100, 3403.9480000000003, 6.911927700042725),
 (100, 3343.0430000000006, 8.43049931526184),
 (200, 3518.6995000000006, 22.15975546836853),
 (200, 2735.8544999999995, 38.66486883163452),
 (300, 2885.661333333333, 55.692084312438965),
 (300, 3000.9323333333323, 59.080138206481934),
 (100, 4947.895999999999, 5.121370315551758),
 (100, 4338.926, 7.072627782821655),
 (200, 5000.6945000000005, 20.88174080848694),
 (200, 3332.981000000001, 30.195629119873047),
 (300, 4031.8820000000005, 53.089977741241455),
 (300, 3247.9799999999996, 57.870660066604614),
 ('mean', 100, 3865.8814999999995),
 ('mean', 200, 3462.954916666666),
 ('mean', 300, 3340.819)]

In [37]:
with open('algorithm_basic_greedy.txt', 'w') as file:
    for number in avg_cost:
        file.write(f"{number}\n")

## 문제 상황 그래프 

In [ ]:
for i in range(10,18):
    
    problem_file = f'./stage1_problem/STAGE1_{i+1}.json'
    
    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']
    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    plt.subplots(figsize=(8, 8))
    node_size = 5

    shop_x = [order.shop_lon for order in ALL_ORDERS]
    shop_y = [order.shop_lat for order in ALL_ORDERS]
    plt.scatter(shop_x, shop_y, c='red', s=node_size, label='SHOPS')

    dlv_x = [order.dlv_lon for order in ALL_ORDERS]
    dlv_y = [order.dlv_lat for order in ALL_ORDERS]
    plt.scatter(dlv_x, dlv_y, c='blue', s=node_size, label='DLVS')
    # 매장과 배송지를 잇는 선을 그립니다.
    for sx, sy, dx, dy in zip(shop_x, shop_y, dlv_x, dlv_y):
        plt.plot([sx, dx], [sy, dy], 'gray', linestyle='-', linewidth=0.5)  # 회색 선으로 매장과 배송지를 연결
